> Submission for https://www.kaggle.com/competitions/spaceship-titanic/overview

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [3]:
train_df = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [4]:
train_df.groupby('Transported').Transported.count() / train_df.Transported.count()

Transported
False    0.496376
True     0.503624
Name: Transported, dtype: float64

### Filling NaN values

In [5]:
train_df.count()

PassengerId     8693
HomePlanet      8492
CryoSleep       8476
Cabin           8494
Destination     8511
Age             8514
VIP             8490
RoomService     8512
FoodCourt       8510
ShoppingMall    8485
Spa             8510
VRDeck          8505
Name            8493
Transported     8693
dtype: int64

In [6]:
# train_df = train_df[~train_df.PassengerId.isnull()]
# train_df = train_df[~train_df.VIP.isnull()]
# train_df = train_df[~train_df.CryoSleep.isnull()]

In [7]:
# train_df = train_df.dropna(subset=['Cabin'])
# train_df

In [8]:
def get_na_null_counts(funct_df):
    return pd.DataFrame(data={
        'isNa': funct_df.isna().sum(),
        'total': funct_df.count(),
        'portion_missing': funct_df.isna().sum()/funct_df.count(),
        'type': funct_df.dtypes
    })

def get_null_counts(funct_df):
    return pd.DataFrame(data={
        'isNa': funct_df.isnull().sum(),
        'total': funct_df.count(),
        'portion_missing': funct_df.isnull().sum()/funct_df.count(),
        'type': funct_df.dtypes
    })

get_na_null_counts(train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.000000,object
HomePlanet,201,8492,0.023669,object
CryoSleep,217,8476,0.025602,object
Cabin,199,8494,0.023428,object
Destination,182,8511,0.021384,object
Age,179,8514,0.021024,float64
VIP,203,8490,0.023910,object
RoomService,181,8512,0.021264,float64
FoodCourt,183,8510,0.021504,float64
ShoppingMall,208,8485,0.024514,float64


In [9]:
train_df_nonna = train_df.copy()
train_df_nonna['HomePlanet'] = train_df.HomePlanet.fillna(train_df.HomePlanet.mode()[0])
train_df_nonna['CryoSleep'] = train_df.CryoSleep.fillna(train_df.CryoSleep.mode()[0])
train_df_nonna['Destination'] = train_df.Destination.fillna(train_df.Destination.mode()[0])
train_df_nonna['VIP'] = train_df.VIP.fillna(train_df.VIP.mode()[0])

train_df_nonna['Age'] = train_df.Age.fillna(train_df.Age.median())
train_df_nonna['RoomService'] = train_df.RoomService.fillna(train_df.RoomService.median())
train_df_nonna['FoodCourt'] = train_df.FoodCourt.fillna(train_df.FoodCourt.median())
train_df_nonna['ShoppingMall'] = train_df.ShoppingMall.fillna(train_df.ShoppingMall.median())
train_df_nonna['Spa'] = train_df.Spa.fillna(train_df.Spa.median())
train_df_nonna['VRDeck'] = train_df.VRDeck.fillna(train_df.VRDeck.median())
train_df_nonna['Cabin'] = train_df.Cabin.fillna('X/0/X')

train_df_nonna['CryoSleep'] = train_df_nonna['CryoSleep'] * 1
train_df_nonna['VIP'] = train_df_nonna.VIP * 1
train_df_nonna['Transported'] = train_df.Transported.astype(int)

train_df_nonna = train_df_nonna.drop('Name', axis=1)

train_df_nonna

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0,A/98/P,55 Cancri e,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0
8689,9278_01,Earth,1,G/1499/S,PSO J318.5-22,18.0,0,0.0,0.0,0.0,0.0,0.0,0
8690,9279_01,Earth,0,G/1500/S,TRAPPIST-1e,26.0,0,0.0,0.0,1872.0,1.0,0.0,1
8691,9280_01,Europa,0,E/608/S,55 Cancri e,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0


In [10]:
get_na_null_counts(train_df_nonna)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
HomePlanet,0,8693,0.0,object
CryoSleep,0,8693,0.0,int64
Cabin,0,8693,0.0,object
Destination,0,8693,0.0,object
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64


In [11]:
train_df_nonna[train_df_nonna.isna().any(axis=1)]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported


In [12]:
train_df_nonna['CryoSleep'] = train_df.CryoSleep.fillna(train_df.CryoSleep.mode()[0])
get_na_null_counts(train_df_nonna)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
HomePlanet,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Cabin,0,8693,0.0,object
Destination,0,8693,0.0,object
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64


### Check if Passenger group has anything to do with Transported

In [13]:
def get_group_and_pid(row):
    split_out = row.PassengerId.split('_')
    row['group'] = split_out[0]
    row['pp'] = split_out[1]
    
    try:
    #     print(row.Cabin)
        split_out = row.Cabin.split('/')
    #     print(split_out)
        row['cabin_deck'] = split_out[0]
        row['cabin_num'] = split_out[1]
        row['cabin_side'] = split_out[2]
    except Exception as e:
        print(row)
        print(e)
    
    return row
    
new_train_df = train_df_nonna.apply(get_group_and_pid, axis=1)
new_train_df = new_train_df.drop('Cabin', axis=1)
new_train_df.head()

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_deck,cabin_num,cabin_side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,0,0001,01,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,1,0002,01,F,0,S
2,0003_01,Europa,False,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,0003,01,A,0,S
3,0003_02,Europa,False,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,0003,02,A,0,S
4,0004_01,Earth,False,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,1,0004,01,F,1,S


In [14]:
get_na_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
HomePlanet,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Destination,0,8693,0.0,object
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64


In [15]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
HomePlanet,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Destination,0,8693,0.0,object
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64


In [16]:
new_train_df[new_train_df.VIP.isnull()]

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_deck,cabin_num,cabin_side


In [17]:
corr = new_train_df[[
    'group', 'Transported'
]].corr()
corr

,group,Transported
group,1.000000,0.021491
Transported,0.021491,1.000000


In [18]:
backup = new_train_df.copy()

In [19]:
new_train_df = backup.copy()

> Therefore we can conclude that group has no relations with Transported.

### Convert Categorical variables to OneHot

In [20]:
def convert_cat_to_onehot(col, new_train_df):
#     print(new_train_df)
    enc = OneHotEncoder().fit(new_train_df[[col]])
    transformed = enc.transform(new_train_df[[col]]).toarray()
    transformed_df = pd.DataFrame(
        transformed,
        columns=[f"{col}_{i}" for i in enc.categories_[0]]
    )
    new_train_df.dropna(subset=[col])
    new_train_df = pd.concat([new_train_df, transformed_df], axis=1)
    new_train_df = new_train_df.drop(col, axis=1)
    return new_train_df

In [21]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
HomePlanet,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Destination,0,8693,0.0,object
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64


In [22]:
print(new_train_df.HomePlanet.unique())
print(new_train_df.Destination.unique())
print(new_train_df.cabin_deck.unique())
print(new_train_df.cabin_side.unique())

['Europa' 'Earth' 'Mars']
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e']
['B' 'F' 'A' 'G' 'X' 'E' 'D' 'C' 'T']
['P' 'S' 'X']


In [23]:
# enc = OneHotEncoder().fit(new_train_df[['cabin_deck']].values)
# enc.categories_

In [24]:
new_train_df = convert_cat_to_onehot('HomePlanet', new_train_df)
new_train_df = convert_cat_to_onehot('Destination', new_train_df)
new_train_df = convert_cat_to_onehot('cabin_deck', new_train_df)
new_train_df = convert_cat_to_onehot('cabin_side', new_train_df)

In [25]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64
VRDeck,0,8693,0.0,float64
Transported,0,8693,0.0,int64


In [26]:
# new_train_df.dropna(['PassengerId'], axis=0)
# new_train_df = new_train_df.dropna(subset=['PassengerId','HomePlanet_Earth'])
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64
VRDeck,0,8693,0.0,float64
Transported,0,8693,0.0,int64


In [27]:
new_train_df.columns

Index(['PassengerId', 'CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt',
       'ShoppingMall', 'Spa', 'VRDeck', 'Transported', 'group', 'pp',
       'cabin_num', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars',
       'Destination_55 Cancri e', 'Destination_PSO J318.5-22',
       'Destination_TRAPPIST-1e', 'cabin_deck_A', 'cabin_deck_B',
       'cabin_deck_C', 'cabin_deck_D', 'cabin_deck_E', 'cabin_deck_F',
       'cabin_deck_G', 'cabin_deck_T', 'cabin_deck_X', 'cabin_side_P',
       'cabin_side_S', 'cabin_side_X'],
      dtype='object')

In [28]:
# new_train_df.Destination_nan.unique()

In [29]:
# new_train_df = new_train_df.drop(
#     ['cabin_side_nan', 'Destination_nan', 'cabin_deck_nan'],
#     axis=1
# )

### Correlations

In [30]:
new_train_df.head()

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,...,cabin_deck_C,cabin_deck_D,cabin_deck_E,cabin_deck_F,cabin_deck_G,cabin_deck_T,cabin_deck_X,cabin_side_P,cabin_side_S,cabin_side_X
0,0001_01,False,39.0,0,0.0,0.0,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0002_01,False,24.0,0,109.0,9.0,25.0,549.0,44.0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0003_01,False,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0003_02,False,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0004_01,False,16.0,0,303.0,70.0,151.0,565.0,2.0,1,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [31]:
get_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64
VRDeck,0,8693,0.0,float64
Transported,0,8693,0.0,int64


In [32]:
get_na_null_counts(new_train_df)

,isNa,total,portion_missing,type
PassengerId,0,8693,0.0,object
CryoSleep,0,8693,0.0,bool
Age,0,8693,0.0,float64
VIP,0,8693,0.0,int64
RoomService,0,8693,0.0,float64
FoodCourt,0,8693,0.0,float64
ShoppingMall,0,8693,0.0,float64
Spa,0,8693,0.0,float64
VRDeck,0,8693,0.0,float64
Transported,0,8693,0.0,int64


In [33]:
corr = new_train_df.corr()
corr.style.background_gradient(cmap='coolwarm')

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,group,pp,cabin_num,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,cabin_deck_A,cabin_deck_B,cabin_deck_C,cabin_deck_D,cabin_deck_E,cabin_deck_F,cabin_deck_G,cabin_deck_T,cabin_deck_X,cabin_side_P,cabin_side_S,cabin_side_X
PassengerId,1.000000,-0.006883,-0.009099,0.013608,0.000375,-0.009200,0.017795,-0.005198,0.015945,0.021491,1.000000,0.011174,0.665621,0.002833,0.004946,-0.008802,-0.001458,0.013533,-0.007238,0.008574,-0.006284,0.012612,-0.008469,-0.000176,0.005232,0.001691,-0.016011,-0.026983,0.010562,-0.002486,-0.026983
CryoSleep,-0.006883,1.000000,-0.071323,-0.078281,-0.244089,-0.205928,-0.207798,-0.198307,-0.192721,0.460132,-0.006883,0.063291,-0.040133,-0.107231,0.093395,0.032715,0.065589,0.087764,-0.113380,-0.030594,0.124049,0.028433,-0.066678,-0.110644,-0.220150,0.241399,-0.017579,0.023356,-0.023858,0.016861,0.023356
Age,-0.009099,-0.071323,1.000000,0.091863,0.068629,0.127390,0.033148,0.120946,0.099590,-0.074233,-0.009099,-0.131422,-0.127788,-0.200814,0.217336,0.015831,0.016817,-0.028121,0.002832,0.073909,0.103305,0.126270,0.077004,0.028212,-0.018397,-0.209928,0.013734,0.003365,-0.011621,0.010609,0.003365
VIP,0.013608,-0.078281,0.091863,1.000000,0.056566,0.125499,0.018412,0.060991,0.123061,-0.037261,0.013608,0.000703,-0.096811,-0.162345,0.147008,0.043523,0.045167,-0.000592,-0.039617,0.132574,0.065080,0.065591,0.067678,-0.012912,-0.057578,-0.098863,-0.003672,0.007429,0.008798,-0.011017,0.007429
RoomService,0.000375,-0.244089,0.068629,0.056566,1.000000,-0.015126,0.052337,0.009244,-0.018624,-0.241124,0.000375,-0.021750,-0.012673,-0.140483,-0.066635,0.245222,-0.023441,-0.063354,0.060680,-0.022937,-0.064536,-0.019494,0.160699,0.043935,0.085627,-0.145289,0.007525,0.026508,0.006991,-0.014918,0.026508
FoodCourt,-0.009200,-0.205928,0.127390,0.125499,-0.015126,1.000000,-0.013717,0.221468,0.224572,0.045583,-0.009200,0.023265,-0.177197,-0.215061,0.365102,-0.124755,0.134952,-0.065400,-0.078269,0.119327,0.155351,0.257511,0.020128,-0.038889,-0.131988,-0.149911,0.014267,0.000160,-0.019682,0.019625,0.000160
ShoppingMall,0.017795,-0.207798,0.033148,0.018412,0.052337,-0.013717,1.000000,0.014542,-0.007849,0.009391,0.017796,-0.038582,0.003530,-0.071911,-0.021155,0.111654,-0.013739,-0.031245,0.031855,-0.017335,-0.010750,0.000906,0.054823,0.031747,0.074473,-0.108538,-0.006787,-0.009738,0.020940,-0.018018,-0.009738
Spa,-0.005198,-0.198307,0.120946,0.060991,0.009244,0.221468,0.014542,1.000000,0.147658,-0.218545,-0.005198,0.016767,-0.129222,-0.159120,0.266103,-0.087989,0.075096,-0.053303,-0.032897,0.060152,0.114257,0.163670,0.033486,-0.011736,-0.087538,-0.124342,0.036316,0.026119,-0.005700,-0.002116,0.026119
VRDeck,0.015945,-0.192721,0.099590,0.123061,-0.018624,0.224572,-0.007849,0.147658,1.000000,-0.204874,0.015945,0.009686,-0.133074,-0.154064,0.282556,-0.111863,0.089667,-0.044861,-0.051119,0.096177,0.122644,0.206639,-0.003542,-0.017905,-0.094548,-0.127434,0.012369,-0.007138,0.009089,-0.006950,-0.007138
Transported,0.021491,0.460132,-0.074233,-0.037261,-0.241124,0.045583,0.009391,-0.218545,-0.204874,1.000000,0.021491,0.066390,-0.043832,-0.168845,0.176916,0.019544,0.108722,0.000092,-0.096319,-0.002623,0.144733,0.108193,-0.034046,-0.097965,-0.087753,0.016269,-0.014568,-0.000340,-0.101397,0.101455,-0.000340


### Multicollinear Checks

In [34]:
# X.values.shape, X.columns, len(X.columns)

In [35]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = new_train_df.copy()
X = X.drop('Transported', axis=1)
X1 = np.array(X.values, dtype=float)
# vif = [
#     variance_inflation_factor(X1, i) for i in range(len(X.columns))
# ]

In [36]:
vif = pd.DataFrame()
vif["features"] = X.columns
vif["vif_Factor"] = [variance_inflation_factor(X1, i) for i in range(X.shape[1])]
vif

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,PassengerId,inf
1,CryoSleep,1.617493
2,Age,1.115224
3,VIP,1.067660
4,RoomService,1.170254
5,FoodCourt,1.284071
6,ShoppingMall,1.074890
7,Spa,1.157659
8,VRDeck,1.176637
9,group,inf


In [37]:
def drop_col_and_get_vif(X, col_name):
#     X = X.drop('cabin_deck_A', axis=1)
    X.drop(col_name, axis=1, inplace=True)
    X1 = np.array(X.values, dtype=float)
    vif = [
        variance_inflation_factor(X1, i) for i in range(len(X.columns))
    ]

    vif = pd.DataFrame()
    vif["features"] = X.columns
    vif["vif_Factor"] = [variance_inflation_factor(X1, i) for i in range(X.shape[1])]
    return vif

drop_col_and_get_vif(X, 'pp')

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,PassengerId,6.747320e+10
1,CryoSleep,1.617493e+00
2,Age,1.115224e+00
3,VIP,1.067660e+00
4,RoomService,1.170254e+00
5,FoodCourt,1.284071e+00
6,ShoppingMall,1.074890e+00
7,Spa,1.157659e+00
8,VRDeck,1.176637e+00
9,group,6.747320e+10


In [38]:
# drop_col_and_get_vif(X, 'HomePlanet_Earth')

In [39]:
drop_col_and_get_vif(X, 'PassengerId')

/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/conda/lib/python3.10/site-packages/statsmodels/stats/outliers_influence.py:198: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,vif_Factor
0,CryoSleep,1.617493
1,Age,1.091180
2,VIP,1.067619
3,RoomService,1.170218
4,FoodCourt,1.284020
5,ShoppingMall,1.074026
6,Spa,1.157616
7,VRDeck,1.176517
8,group,4.085255
9,cabin_num,7.051794


In [40]:
# drop_col_and_get_vif(X, 'cabin_side_P')

In [41]:
# drop_col_and_get_vif(X, 'cabin_deck_X')

In [42]:
# drop_col_and_get_vif(X, 'Destination_TRAPPIST-1e')

In [43]:
selected_columns = X.columns
selected_columns

Index(['CryoSleep', 'Age', 'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall',
       'Spa', 'VRDeck', 'group', 'cabin_num', 'HomePlanet_Earth',
       'HomePlanet_Europa', 'HomePlanet_Mars', 'Destination_55 Cancri e',
       'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'cabin_deck_A',
       'cabin_deck_B', 'cabin_deck_C', 'cabin_deck_D', 'cabin_deck_E',
       'cabin_deck_F', 'cabin_deck_G', 'cabin_deck_T', 'cabin_deck_X',
       'cabin_side_P', 'cabin_side_S', 'cabin_side_X'],
      dtype='object')

# Processing Test Dataset

In [69]:
test_df = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
results_df = test_df[['PassengerId']]

test_df['HomePlanet'] = test_df.HomePlanet.fillna(test_df.HomePlanet.mode()[0])
test_df['CryoSleep'] = test_df.CryoSleep.fillna(test_df.CryoSleep.mode()[0])
test_df['Destination'] = test_df.Destination.fillna(test_df.Destination.mode()[0])
test_df['VIP'] = test_df.VIP.fillna(test_df.VIP.mode()[0])

test_df['Age'] = test_df.Age.fillna(test_df.Age.median())
test_df['RoomService'] = test_df.RoomService.fillna(test_df.RoomService.median())
test_df['FoodCourt'] = test_df.FoodCourt.fillna(test_df.FoodCourt.median())
test_df['ShoppingMall'] = test_df.ShoppingMall.fillna(test_df.ShoppingMall.median())
test_df['Spa'] = test_df.Spa.fillna(test_df.Spa.median())
test_df['VRDeck'] = test_df.VRDeck.fillna(test_df.VRDeck.median())
test_df['Cabin'] = test_df.Cabin.fillna('X/0/X')

test_df['CryoSleep'] = test_df['CryoSleep'] * 1
test_df['VIP'] = test_df.VIP * 1



test_df['CryoSleep'] = test_df.CryoSleep.fillna(test_df.CryoSleep.mode()[0])
new_test_df = test_df.apply(get_group_and_pid, axis=1)
new_test_df = new_test_df.drop(['Name', 'PassengerId', 'pp', 'Cabin'], axis=1)

new_test_df = convert_cat_to_onehot('HomePlanet', new_test_df)
new_test_df = convert_cat_to_onehot('Destination', new_test_df)
new_test_df = convert_cat_to_onehot('cabin_deck', new_test_df)
new_test_df = convert_cat_to_onehot('cabin_side', new_test_df)

In [70]:
get_null_counts(new_test_df)

,isNa,total,portion_missing,type
CryoSleep,0,4277,0.0,int64
Age,0,4277,0.0,float64
VIP,0,4277,0.0,int64
RoomService,0,4277,0.0,float64
FoodCourt,0,4277,0.0,float64
ShoppingMall,0,4277,0.0,float64
Spa,0,4277,0.0,float64
VRDeck,0,4277,0.0,float64
group,0,4277,0.0,object
cabin_num,0,4277,0.0,object


In [75]:
def get_prediction_for_test(test_df, model):
    return model.predict(test_df) == 1

# Logistic Regression

In [64]:
from sklearn.linear_model import LogisticRegression

input_df = new_train_df[selected_columns]
output_df = new_train_df['Transported']

clf = LogisticRegression(random_state=0).fit(input_df, output_df)
clf

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [65]:
# input_df[:2]
clf.predict(input_df[:5]), output_df[:5]

(array([1, 0, 0, 0, 0]),
 0    0
 1    1
 2    0
 3    0
 4    1
 Name: Transported, dtype: int64)

In [66]:
clf.score(input_df, output_df)

0.7897158633383182

In [77]:
# new_test_df

In [76]:
results_df['Transported'] = get_prediction_for_test(new_test_df, clf)
results_df

/tmp/ipykernel_299/3840762891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Transported'] = get_prediction_for_test(new_test_df, clf)


,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [85]:
results_df.Transported.describe()

count     4277
unique       2
top       True
freq      2641
Name: Transported, dtype: object

# Logistic Regression - Hyper parameter tuning

In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(new_train_df[selected_columns])
input_df = scaler.transform(new_train_df[selected_columns])
output_df = new_train_df['Transported']

clf = LogisticRegression(random_state=0)\
        .fit(input_df, output_df)
clf.predict(input_df[:5]), output_df[:5]
clf.score(input_df, output_df)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7695847233406189

In [81]:
results_df['Transported'] = get_prediction_for_test(new_test_df, clf)
results_df

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/tmp/ipykernel_299/3840762891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Transported'] = get_prediction_for_test(new_test_df, clf)


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [83]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(new_train_df[selected_columns])
input_df = scaler.transform(new_train_df[selected_columns])
output_df = new_train_df['Transported']

clf = LogisticRegression(random_state=0)\
        .fit(input_df, output_df)
clf.predict(input_df[:5]), output_df[:5]
clf.score(input_df, output_df)

0.7929368457379501

In [84]:
results_df['Transported'] = get_prediction_for_test(new_test_df, clf)
results_df

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/tmp/ipykernel_299/3840762891.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df['Transported'] = get_prediction_for_test(new_test_df, clf)


,PassengerId,Transported
0,0013_01,False
1,0018_01,False
2,0019_01,False
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,True
4274,9271_01,True
4275,9273_01,True


In [88]:
results_df.to_csv('/kaggle/working/submission.csv', index=False)